In [14]:
import re
import pandas as pd
from dataclasses import dataclass
import xml.etree.ElementTree as ET

""" This script reads reviews from the Amazon Review Dataset and parses them into a list of Review objects. """

@dataclass
class Review:
    is_negative: int
    asin: str
    product_name: str
    product_type: str
    helpful: str
    rating: float
    title: str
    date: str
    reviewer: str
    reviewer_location: str
    review_text: str


class ReviewXMLParser:

    @staticmethod
    def get_element_text(element, tag, default=''):
        # Handle namespaces if present
        namespaces = {'ns': element.tag.split('}')[0].strip('{')} if '}' in element.tag else {}
        found_element = element.find(f'.//{tag}', namespaces)
        return found_element.text if found_element is not None else default

    @staticmethod
    def parse(xml, negative=None) -> list[Review]:
        root = ET.fromstring(xml)
        reviews = []
        for review_element in root.findall(".//review"):

            rating = float(ReviewXMLParser.get_element_text(review_element, "rating"))

            review = Review(
                is_negative=negative,  # Assuming all reviews in negative_reviews are negative
                asin=ReviewXMLParser.get_element_text(review_element, "asin").strip(),
                product_name=ReviewXMLParser.get_element_text(review_element, "product_name"),
                product_type=ReviewXMLParser.get_element_text(review_element, "product_type"),
                helpful=ReviewXMLParser.get_element_text(review_element, "helpful"),
                rating=rating,
                title=ReviewXMLParser.get_element_text(review_element, "title"),
                date=ReviewXMLParser.get_element_text(review_element, "date"),
                reviewer=ReviewXMLParser.get_element_text(review_element, "reviewer"),
                reviewer_location=ReviewXMLParser.get_element_text(review_element, "reviewer_location"),
                review_text=ReviewXMLParser.get_element_text(review_element, "review_text"),
            )
            reviews.append(review)
        return reviews

reviews = []

review_files = [
    # Positive reviews
    ("sorted_data_acl/books/negative.review", 1),
    ("sorted_data_acl/dvd/negative.review", 1),
    ("sorted_data_acl/electronics/negative.review", 1),
    ("sorted_data_acl/kitchen_housewares/negative.review", 1),
    
    # Negative reviews 
    ("sorted_data_acl/books/positive.review", 0),
    ("sorted_data_acl/dvd/positive.review", 0),
    ("sorted_data_acl/electronics/positive.review", 0),
    ("sorted_data_acl/kitchen_housewares/positive.review", 0),

    # Unlabeled reviews
    ("sorted_data_acl/dvd/unlabeled.review", None),
    ("sorted_data_acl/electronics/unlabeled.review", None),
    ("sorted_data_acl/kitchen_housewares/unlabeled.review", None),
]

for file, is_negative in review_files:
    with open(file) as f:
        print(f"Reading {file}")    
        normalized = f.read()
        # normalized = re.sub(r'[^<->-/-A-Za-z0-9-\s]+', '', normalized)
        normalized = re.sub(r'&quot', '', normalized)
        normalized = re.sub(r'&', '', normalized)
        normalized = re.sub(r'', '', normalized)
        normalized = re.sub(r'\?', '', normalized)
        normalized = re.sub(r'\n', ' ', normalized)
        normalized = re.sub(r'"', '\'', normalized)

        # print(normalized)
        reviews += ReviewXMLParser.parse(normalized, is_negative)

df = pd.DataFrame(reviews)

df_transformed = df.copy()
df

Reading sorted_data_acl/books/negative.review
Reading sorted_data_acl/dvd/negative.review
Reading sorted_data_acl/electronics/negative.review
Reading sorted_data_acl/kitchen_housewares/negative.review
Reading sorted_data_acl/books/positive.review
Reading sorted_data_acl/dvd/positive.review
Reading sorted_data_acl/electronics/positive.review
Reading sorted_data_acl/kitchen_housewares/positive.review
Reading sorted_data_acl/dvd/unlabeled.review
Reading sorted_data_acl/electronics/unlabeled.review
Reading sorted_data_acl/kitchen_housewares/unlabeled.review


,is_negative,asin,product_name,product_type,helpful,rating,title,date,reviewer,reviewer_location,review_text
0,1.0,0312355645,Running with Scissors: A Memoir: Books: Augus...,books,4 of 9,1.0,"Horrible book, horrible.","November 14, 2006",Mark Gospri,,THis book was horrible. If it was possible t...
1,1.0,1559278676,Running with Scissors: A Memoir: Books: Augus...,books,1 of 6,1.0,shallow self-indulgence,"November 15, 2006",Joseph S. Perrott 'avid reader',"philadelphia, pa. United States",I like to use the Amazon reviews when purchas...
2,1.0,1559278676,Running with Scissors: A Memoir: Books: Augus...,books,4 of 9,1.0,"Horrible book, horrible.","November 14, 2006",Mark Gospri,,THis book was horrible. If it was possible t...
3,1.0,0425193373,Fierce Conversations: Achieving Sucess at Wor...,books,10 of 13,1.0,Disappointment,"March 13, 2006",Reader 'Reader',,"I'm not sure who's writing these reviews, but..."
4,1.0,0142004030,Lost in a Good Book (Thursday Next Novels): B...,books,6 of 7,2.0,A Disappointing Mess,"May 17, 2006",A. Ross,"Washington, DC",I picked up the first book in this series (Th...
...,...,...,...,...,...,...,...,...,...,...,...
72674,NaN,B00006IUU0,Mr. Coffee Cocomotion Hot Chocolate Maker: Ki...,kitchen housewares,,5.0,It's cold outside!,"September 29, 2006",Adam G,"Sonoma County, CA USA",What a perfect little machine! I love my Coco...
72675,NaN,B00006IUU0,Mr. Coffee Cocomotion Hot Chocolate Maker: Ki...,kitchen housewares,,5.0,Great Gift!,"February 27, 2006",M. Strouse 'pezhead859',"Brooklyn, New York",I gave this as a gift and was told this makes...
72676,NaN,B00006IUU0,Mr. Coffee Cocomotion Hot Chocolate Maker: Ki...,kitchen housewares,1 of 1,5.0,A must have for coffee and hot drink users al...,"February 25, 2006",C. Trentham,"Plano, Texas USA",We got this as a Christmas gift in December a...
72677,NaN,B00006IUU0,Mr. Coffee Cocomotion Hot Chocolate Maker: Ki...,kitchen housewares,4 of 5,5.0,Great little work horse that Makes more thing...,"February 18, 2006",Dasher 'marbleann',"Huntington, NY United States",I got this little machine when it first came ...


In [15]:
df.shape

(72679, 11)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72679 entries, 0 to 72678
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   is_negative        8000 non-null   float64
 1   asin               72679 non-null  object 
 2   product_name       72679 non-null  object 
 3   product_type       72679 non-null  object 
 4   helpful            72679 non-null  object 
 5   rating             72679 non-null  float64
 6   title              72679 non-null  object 
 7   date               72679 non-null  object 
 8   reviewer           72679 non-null  object 
 9   reviewer_location  72679 non-null  object 
 10  review_text        72679 non-null  object 
dtypes: float64(2), object(9)
memory usage: 6.1+ MB


In [17]:
""" Normalize `helpful` column """

# It would be possible to transform a `x of y` to a % proportion between 0 to 1

df_transformed['helpful'] = df['helpful'].str.replace(' of ', ';').str.replace(' ', '')

def transform_helpful(helpful):
    if helpful == '':
        return 0
    x, y = helpful.split(';')
    x = int(x)
    y = int(y)
    if x == 0:
        return 0
    
    return int(x) / int(y)

df_transformed['helpful'] = df_transformed['helpful'].apply(transform_helpful)
df_transformed['helpful'].value_counts()

helpful
1.000000    29582
0.000000    17840
0.500000     3676
0.666667     2607
0.750000     1820
            ...  
0.564103        1
0.542857        1
0.952830        1
0.228571        1
0.981132        1
Name: count, Length: 723, dtype: int64

In [18]:
""" Transform text reviews to lowercase """

from textblob import TextBlob


def transform_text(helpful):
    blob = TextBlob(helpful)
    blob = blob.lower()
    # blob = blob.correct()
    return str(blob)

df_transformed['review_text'] = df['review_text'].apply(transform_text)

In [19]:
df_transformed[['review_text', 'is_negative']]

,review_text,is_negative
0,this book was horrible. if it was possible t...,1.0
1,i like to use the amazon reviews when purchas...,1.0
2,this book was horrible. if it was possible t...,1.0
3,"i'm not sure who's writing these reviews, but...",1.0
4,i picked up the first book in this series (th...,1.0
...,...,...
72674,what a perfect little machine! i love my coco...,NaN
72675,i gave this as a gift and was told this makes...,NaN
72676,we got this as a christmas gift in december a...,NaN
72677,i got this little machine when it first came ...,NaN


In [20]:
df_transformed[['review_text', 'is_negative']].to_csv('reviews.csv', index=False)